## 模型保存

模型训练完成之后导出的checkpoint，可通过deploy接口将模型保存到云平台进行管理，返回一个模型引用的唯一tag，并在部署到云平台AI解译工具箱时可根据模型tag重新初始化checkpoint，自动将模型下载到本地cache目录

### 获取授权

在与云平台进行交互时，需要先进行云平台的授权，获取用户个人token才可使用

In [ ]:
from aiearth.core import Authenticate

Authenticate()

### 创建项目空间

项目空间和模型任务一一对应，表示一类模型任务，并通过 project_id 进行索引

In [ ]:
from aiearth.predict.deploy.sdk import Project

project = Project.create("自定义模型pipeline")
print(project)

### 创建模型版本

每个项目空间下面，可以有多个模型版本，模型版本命名规则请遵循 x.y.z，xyz均为数字，如1.0.1，并通过 version_id 进行索引

In [ ]:
from aiearth.predict.deploy.sdk import Version

version = project.create_version("1.0.1")
print(version)

### 模型保存

从本地模型的onnx路径初始化ModelCheckpoint，并添加TeslaT4上转换的Tensorrt engine文件作为模型附属文件，将模型保存到云平台，并返回模型引用的`saved_model_tag`

`ModelCheckpoint.from_local_path`的path参数为本地onnx的模型路径，image_size参数是模型输入图像的大小，bound参数是模型预测需要忽略的边界大小，不计入最后结果，attached_files参数是模型其他附属文件，这里将加速之后的trt文件作为附属文件`attached_files`进行了上传，`attached_files`参数类型为`Dict`，其中key为onnx模型文件名（去除后缀）+ 显卡类型 + trt engine后缀，在推理时`TensorrtPredictor`可根据模型文件名索引对应的trt engine，value类型为`ModelAttachedFile`，`path`参数为本地对应的trt文件路径

In [ ]:
from aiearth.predict.checkpoint import (
    ModelCheckpoint,
    ModelAttachedFile,
    AcceleratorType,
)

ckpt = ModelCheckpoint.from_local_path(
    path="/path/to/your/fake_name.onnx",
    image_size=1024,
    bound=128,
    attached_files={
        "fake_name.TeslaT4.trt": ModelAttachedFile(
            path="/path/to/your/fake_name.TeslaT4.trt",
            accelerator_type=AcceleratorType.t4,
        )
    },
)

saved_model_tag = version.save_model(ckpt)
print(saved_model_tag)

运行的时候需要将`/path/to/your/fake_name.onnx`替换为本地onnx模型的绝对路径，`fake_name.TeslaT4.trt`中的`fake_name`替换为上传的模型文件名，以及`ModelAttachedFile`的`path`参数替换为本地对应的trt文件绝对路径

### 模型引用

保存到云平台的模型，可通过模型的tag重新初始化checkpoint，自动将模型下载到本地的cache目录

In [ ]:
import json

ckpt = ModelCheckpoint.from_remote_tag(saved_model_tag)
print(json.dumps(ckpt.get_cfg().dict()))

### 模型列表

保存到云平台的模型是按照项目空间和模型版本进行管理的，可以查询当前模型版本下已保存的模型信息

In [ ]:
import json

models = version.list_saved_models()
print(json.dumps(models))